In [1]:
deployment = "gpt4"
model = "gpt-4"

# IaC：生成 Terraform 脚本

In [2]:
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

llm = AzureChatOpenAI(deployment_name=deployment, temperature=0.3, max_tokens=1000,
                     streaming=True)

memory = ConversationBufferWindowMemory(k=10) 


def get_response(input):
    conversation_with_memory = ConversationChain(
        llm=llm, 
        memory=memory,
        verbose=False
    )
    return conversation_with_memory.predict(input=input)

In [3]:
print(get_response("""
给我一个terraform脚本示例，用来创建一个AWS EKS集群
"""))

很抱歉，由于我是一个文本生成的AI，我无法直接提供代码或脚本。但是我可以告诉你大概的步骤和你可能需要的资源。

首先，你需要一个Terraform的配置文件。这个文件会定义你要创建的资源，比如EKS集群，以及这些资源的配置。

你可以参考以下的基本结构来创建你的Terraform脚本：

```hcl
provider "aws" {
  region = "us-west-2"
}

module "eks" {
  source          = "terraform-aws-modules/eks/aws"
  cluster_name    = "my-eks-cluster"
  cluster_version = "1.17"
  subnets         = ["subnet-abcde012", "subnet-bcde012a", "subnet-fghi345a"]
  vpc_id          = "vpc-abcde012"

  node_groups = {
    eks_nodes = {
      desired_capacity = 2
      max_capacity     = 10
      min_capacity     = 1

      instance_type = "m4.large"
      key_name      = "my-key-name"
    }
  }
}
```

这个脚本会创建一个名为"my-eks-cluster"的EKS集群，版本为1.17。集群会在指定的VPC和子网中创建，并且会创建一个名为"eks_nodes"的节点组，这个节点组的实例类型为"m4.large"，并且会使用名为"my-key-name"的密钥对。

请注意，这只是一个基本的示例，你可能需要根据你的实际需求来修改这个脚本。例如，你可能需要添加更多的配置选项，或者创建更多的资源。

在你运行这个脚本之前，你需要确保你已经安装了Terraform，并且你的AWS账户已经有了足够的权限来创建这些资源。你也需要确保你已经设置了正确的AWS凭证，Terraform可以使用这些凭证来访问你的AWS账户。

你可以在Terraform的官方文档中找到更多关于如何使用Terraform来创建AWS资源的

In [4]:
print(get_response("""
将上面的script改写为适合阿里云的
"""))

在阿里云上创建Kubernetes集群，你需要使用阿里云的Terraform提供者。以下是一个基本的Terraform脚本示例，用于在阿里云上创建一个Kubernetes集群：

```hcl
provider "alicloud" {
  access_key = "<your-access-key>"
  secret_key = "<your-secret-key>"
  region     = "cn-hangzhou"
}

resource "alicloud_cs_managed_kubernetes" "k8s" {
  name_prefix           = "my-k8s-cluster"
  worker_instance_types = ["ecs.g5.large"]
  worker_number         = 2
  password              = "<your-password>"
  pod_cidr              = "172.20.0.0/16"
  service_cidr          = "172.21.0.0/20"
  install_cloud_monitor = true
}
```

这个脚本会创建一个名为"my-k8s-cluster"的Kubernetes集群，工作节点的实例类型为"ecs.g5.large"，工作节点的数量为2。集群的Pod CIDR为"172.20.0.0/16"，Service CIDR为"172.21.0.0/20"，并且会安装云监控服务。

请注意，你需要替换`<your-access-key>`、`<your-secret-key>`和`<your-password>`为你自己的阿里云Access Key、Secret Key和密码。

同样，这只是一个基本的示例，你可能需要根据你的实际需求来修改这个脚本。例如，你可能需要添加更多的配置选项，或者创建更多的资源。

在你运行这个脚本之前，你需要确保你已经安装了Terraform，并且你的阿里云账户已经有了足够的权限来创建这些资源。你也需要确保你已经设置了正确的阿里云凭证，Terraform可以使用这些凭证来访问你的阿里云账户。

你可以在Terraform的官方文档中找到更多关于如何使

# 生成应用部署文件

In [5]:
import openai
def work_on(input):
    response = openai.ChatCompletion.create(
        engine=deployment, # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a senior software engineer."},   
            {"role": "user", "content": input}
        ],
        temperature = 0.9, 
        max_tokens = 1000,
      )
    return response.choices[0].message.content

In [6]:
print(work_on("""
编写kubernetes部署文件（yaml）用于部署Nginx服务，服务端口8080
满足以下的要求
1. 部署的namespace为”service“
2. 包含readiness及liveness probe
3. 采用一个独立的service account运行
4. 包含完美终止（graceful termination）配置
"""))

以下是一个满足您需求的 Kubernetes 服务部署 yaml 文件的示例:

```yaml
apiVersion: v1
kind: Namespace
metadata:
  name: service

---
apiVersion: v1
kind: ServiceAccount
metadata:
  name: nginx-service-account
  namespace: service

---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  namespace: service
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      serviceAccountName: nginx-service-account
      containers:
      - name: nginx
        image: nginx:1.14.2
        ports:
        - containerPort: 80
        readinessProbe:
          httpGet:
            path: /
            port: 80
          initialDelaySeconds: 5
          periodSeconds: 5
        livenessProbe:
          httpGet:
            path: /
            port: 80
          initialDelaySeconds: 15
          periodSeconds: 20
        lifecycle:
          preStop:
            exec:
              command: ["/bin/sh","-c","nginx -s quit; wh

In [7]:
from langchain import PromptTemplate, OpenAI, LLMChain

from langchain.chat_models import AzureChatOpenAI
# from langchain.chat_models import ChatOpenAI #直接访问OpenAI的GPT服务

def create_k8s_config(requirements):
    prompt_template = """
    根据以下用户需求：
    "{requirements}"
    编写kubernetes部署文件（yaml), 
    满足以下的要求
    1. 部署的namespace为”service“
    2. 包含readiness及liveness probe
    3. 采用一个独立的service account运行
    4. 包含完美终止（graceful termination）配置
    """

    #llm = ChatOpenAI(model_name="gpt-4", temperature=0) #直接访问OpenAI的GPT服务
    llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200) # 通过Azure的OpenAI服务
    llm_chain = LLMChain(
        llm=llm,
        prompt=PromptTemplate.from_template(prompt_template)
    )
    return llm_chain.run(requirements)

In [8]:
print(create_k8s_config("部署Nginx服务，服务端口8080"))

以下是一个满足上述要求的Kubernetes部署文件示例：

```yaml
apiVersion: v1
kind: Namespace
metadata:
  name: service
---
apiVersion: v1
kind: ServiceAccount
metadata:
  name: nginx-service-account
  namespace: service
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  namespace: service
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      serviceAccountName: nginx-service-account
      containers:
      - name: nginx
        image: nginx:1.14.2
        ports:
        - containerPort: 8080
        readinessProbe:
          httpGet:
            path: /
            port: 8080
          initialDelaySeconds: 5
          period
